# Parafrasear texto

In [6]:
!pip install transformers
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 43.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#  Cargamos libreiras

In [7]:
from transformers import pipeline
import spacy

# Cargar spaCy con un modelo que incluya vectores o suficiente capacidad (se puede usar "es_core_news_md" si está disponible)
nlp = spacy.load("es_core_news_sm")

# Inicializar el pipeline fill-mask para español
fill_mask = pipeline("fill-mask", model="dccuchile/bert-base-spanish-wwm-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Device set to use cpu


# Preparamos las Funciones

In [11]:
def obtener_variaciones_automaticas(palabra, top_k=3):
    """
    Dada una palabra, utiliza un pipeline fill-mask para obtener sinónimos (variaciones).
    Se crea una frase plantilla simple para que el modelo prediga una palabra en el lugar de [MASK].
    """
    # Usamos un template sencillo; el modelo tendrá que inferir el sinónimo en ese contexto.
    template = f"El sinónimo de {palabra} es [MASK]."
    predictions = fill_mask(template, top_k=top_k)
    variaciones = []
    for pred in predictions:
        token = pred['token_str'].strip()
        # Excluir si el token es igual a la palabra original o contiene caracteres no alfabéticos
        if token.lower() != palabra.lower() and token.isalpha():
            variaciones.append(token)
    return variaciones

def generar_variaciones_automaticas(frase):
    """
    Para cada token relevante (VERB, NOUN, ADP) en la frase,
    se obtienen sinónimos automáticamente y se generan nuevas versiones de la frase.
    """
    doc = nlp(frase)
    variaciones = []
    for token in doc:
        if token.pos_ in ["VERB", "NOUN", "ADP"]:
            # Obtener sinónimos automáticos para el token
            sinónimos = obtener_variaciones_automaticas(token.text, top_k=3)
            for sinónimo in sinónimos:
                # Reemplazar solo la primera aparición de la palabra para evitar múltiples cambios
                nueva_frase = frase.replace(token.text, sinónimo, 1)
                variaciones.append(nueva_frase)
    return variaciones

Primer ejemplo

In [9]:
# Ejemplo de uso:
frase_original = "Ella quiere ir de excursión a las montañas este fin de semana."
variaciones_generadas = generar_variaciones_automaticas(frase_original)

print("Variaciones generadas automáticamente:")
for i, v in enumerate(variaciones_generadas, 1):
    print(f"{i}. {v}")

Variaciones generadas automáticamente:
1. Ella querer ir de excursión a las montañas este fin de semana.
2. Ella quiere ir el excursión a las montañas este fin de semana.
3. Ella quiere ir de paseo a las montañas este fin de semana.
4. Ellb quiere ir de excursión a las montañas este fin de semana.
5. Ella quiere ir de excursión a las montaña este fin de semana.
6. Ella quiere ir de excursión a las montañas este infinito de semana.
7. Ella quiere ir el excursión a las montañas este fin de semana.
8. Ella quiere ir de excursión a las montañas este fin de domingo.


Segundo ejemplo

In [12]:
# Ejemplo de uso:
frase_original = "Tengo que actualizar mi computadora con el último software."
variaciones_generadas = generar_variaciones_automaticas(frase_original)

print("Variaciones generadas automáticamente:")
for i, v in enumerate(variaciones_generadas, 1):
    print(f"{i}. {v}")

Variaciones generadas automáticamente:
1. Tener que actualizar mi computadora con el último software.
2. Tengo que actualizar mi computadora con el último soft.


Tercer Ejemplo

In [13]:
# Ejemplo de uso:
frase_original = "Quiero reservar un vuelo a Nueva York el próximo sábado."
variaciones_generadas = generar_variaciones_automaticas(frase_original)

print("Variaciones generadas automáticamente:")
for i, v in enumerate(variaciones_generadas, 1):
    print(f"{i}. {v}")



Variaciones generadas automáticamente:
1. Quiero reservar un mariposa a Nueva York el próximo sábado.
2. Quiero reservbr un vuelo a Nueva York el próximo sábado.
3. Quiero reservar un vuelo a Nueva York el próximo domingo.
4. Quiero reservar un vuelo a Nueva York el próximo viernes.


Ejemplo utilizando Parrot

In [1]:
# 1. Primero instalamos la biblioteca correctamente
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
!pip install -q transformers sentencepiece parrot
!pip install transformers
!pip install torch
!pip install transformers
!pip install spacy
!python -m spacy download es_core_news_sm


  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-13ww07kz
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-13ww07kz
  Resolved https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to commit 03084c54b64019ba5fa0b620b9c70ad81123e458
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 48.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import spacy
from parrot import Parrot
import torch
from transformers import MarianMTModel, MarianTokenizer, pipeline

# --- Cargar modelos ---
# Cargar spaCy para español
nlp = spacy.load("es_core_news_sm")

# Inicializar Parrot (modelo de paráfrasis en inglés)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=torch.cuda.is_available())

# Cargar modelos de traducción
model_name_es_en = "Helsinki-NLP/opus-mt-es-en"
tokenizer_es_en = MarianTokenizer.from_pretrained(model_name_es_en)
model_es_en = MarianMTModel.from_pretrained(model_name_es_en)

model_name_en_es = "Helsinki-NLP/opus-mt-en-es"
tokenizer_en_es = MarianTokenizer.from_pretrained(model_name_en_es)
model_en_es = MarianMTModel.from_pretrained(model_name_en_es)

# Inicializar el pipeline fill-mask para español
fill_mask = pipeline("fill-mask", model="dccuchile/bert-base-spanish-wwm-cased")


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Device set to use cpu


Declaracion de funciones

In [4]:

# --- Funciones de traducción ---
def traducir_es_a_en(texto):
    inputs = tokenizer_es_en(texto, return_tensors="pt", padding=True)
    outputs = model_es_en.generate(**inputs)
    return tokenizer_es_en.decode(outputs[0], skip_special_tokens=True)

def traducir_en_a_es(texto):
    inputs = tokenizer_en_es(texto, return_tensors="pt", padding=True)
    outputs = model_en_es.generate(**inputs)
    return tokenizer_en_es.decode(outputs[0], skip_special_tokens=True)

# --- Funciones para generar variaciones automáticas ---
def obtener_variaciones_automaticas(palabra, top_k=3):
    """
    Usa un template para obtener sinónimos de la palabra usando fill-mask.
    """
    template = f"El sinónimo de {palabra} es [MASK]."
    predictions = fill_mask(template, top_k=top_k)
    variaciones = []
    for pred in predictions:
        token = pred['token_str'].strip()
        if token.lower() != palabra.lower() and token.isalpha():
            variaciones.append(token)
    return variaciones

def generar_variaciones_automaticas(frase):
    """
    Para cada token relevante (VERB, NOUN, ADP) en la frase en español,
    se obtienen sinónimos automáticamente y se generan nuevas versiones de la frase.
    """
    doc = nlp(frase)
    variaciones = []
    for token in doc:
        if token.pos_ in ["VERB", "NOUN", "ADP"]:
            sinónimos = obtener_variaciones_automaticas(token.text, top_k=3)
            for sinónimo in sinónimos:
                # Reemplazar solo la primera ocurrencia del token para evitar cambios excesivos
                nueva_frase = frase.replace(token.text, sinónimo, 1)
                variaciones.append(nueva_frase)
    return variaciones

# --- Función para generar paráfrasis usando Parrot ---
def generar_parafraseos_con_parrot(frase_es):
    """
    Toma una frase en español, la traduce al inglés, genera paráfrasis con Parrot,
    y traduce las paráfrasis resultantes de vuelta a español.
    """
    # Traducir al inglés
    frase_en = traducir_es_a_en(frase_es)
    # Generar paráfrasis con Parrot
    parafraseos = parrot.augment(
        input_phrase=frase_en,
        max_return_phrases=5,
        adequacy_threshold=0.6,
        fluency_threshold=0.6
    )
    resultados = []
    for item in parafraseos:
        if isinstance(item, tuple) and len(item) >= 1:
            texto_en = item[0]
        elif isinstance(item, str):
            texto_en = item
        else:
            continue
        # Traducir la paráfrasis generada de vuelta al español
        texto_es = traducir_en_a_es(texto_en)
        resultados.append(texto_es)
    return resultados



Primer ejemplo

In [5]:

# --- Bloque principal ---
# Frase de entrada en español
frase_es = "Ella quiere ir de excursión a las montañas este fin de semana."

# Generar paráfrasis con Parrot (a través de traducción)
parafraseos_parrot = generar_parafraseos_con_parrot(frase_es)

# Generar variaciones automáticas usando fill-mask
variaciones_automaticas = generar_variaciones_automaticas(frase_es)

# Combinar ambos resultados y eliminar duplicados
todos_resultados = list(set(parafraseos_parrot + variaciones_automaticas))
todos_resultados = todos_resultados[:10]  # Limitar a 10 resultados, por ejemplo

# Mostrar resultados finales
print("\nParáfrasis y variaciones en español:")
for i, r in enumerate(todos_resultados, 1):
    print(f"{i}. {r}")


Paráfrasis y variaciones en español:
1. Quería ir de excursión a las montañas este fin de semana.
2. Ella quiere ir de excursión a las montañas este fin de domingo.
3. ella quiere ir de excursión en las montañas este fin de semana
4. Ella quiere ir el excursión a las montañas este fin de semana.
5. ella quiere caminar a las montañas este fin de semana
6. Ella quiere ir de paseo a las montañas este fin de semana.
7. Ellb quiere ir de excursión a las montañas este fin de semana.
8. Ella quiere ir de excursión a las montañas este infinito de semana.
9. Ella quiere ir de excursión a las montaña este fin de semana.
10. Ella querer ir de excursión a las montañas este fin de semana.


Segundo Ejemplo

In [9]:

# --- Bloque principal ---
# Frase de entrada en español
frase_es = "Tengo que actualizar mi computadora con el último software."

# Generar paráfrasis con Parrot (a través de traducción)
parafraseos_parrot = generar_parafraseos_con_parrot(frase_es)

# Generar variaciones automáticas usando fill-mask
variaciones_automaticas = generar_variaciones_automaticas(frase_es)

# Combinar ambos resultados y eliminar duplicados
todos_resultados = list(set(parafraseos_parrot + variaciones_automaticas))
todos_resultados = todos_resultados[:10]  # Limitar a 10 resultados, por ejemplo

# Mostrar resultados finales
print("\nParáfrasis y variaciones en español:")
for i, r in enumerate(todos_resultados, 1):
    print(f"{i}. {r}")


Paráfrasis y variaciones en español:
1. Tengo que actualizar mi computadora con el último soft.
2. Tener que actualizar mi computadora con el último software.
3. Necesito actualizar mi ordenador con el software más reciente
4. Tengo que actualizar mi ordenador con el último software


Tercer ejemplo

In [8]:

# --- Bloque principal ---
# Frase de entrada en español
frase_es = "Quiero reservar un vuelo a Nueva York el próximo sábado."

# Generar paráfrasis con Parrot (a través de traducción)
parafraseos_parrot = generar_parafraseos_con_parrot(frase_es)

# Generar variaciones automáticas usando fill-mask
variaciones_automaticas = generar_variaciones_automaticas(frase_es)

# Combinar ambos resultados y eliminar duplicados
todos_resultados = list(set(parafraseos_parrot + variaciones_automaticas))
todos_resultados = todos_resultados[:10]  # Limitar a 10 resultados, por ejemplo

# Mostrar resultados finales
print("\nParáfrasis y variaciones en español:")
for i, r in enumerate(todos_resultados, 1):
    print(f"{i}. {r}")



Paráfrasis y variaciones en español:
1. Quiero reservar un vuelo a Nueva York el próximo domingo.
2. Quiero reservar un vuelo a Nueva York el próximo sábado.
3. Quiero reservar un mariposa a Nueva York el próximo sábado.
4. Quiero reservar un vuelo a Nueva York el próximo viernes.
5. Quiero reservbr un vuelo a Nueva York el próximo sábado.


In [1]:
!pip install googletrans==4.0.0-rc1

In [4]:
import spacy
from parrot import Parrot
import torch
from transformers import pipeline
from googletrans import Translator

# --- Configuración de modelos y traductor ---
# Cargar spaCy para español
nlp = spacy.load("es_core_news_sm")

# Inicializar Parrot (para paráfrasis en inglés)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=torch.cuda.is_available())

# Inicializar el traductor de Google
translator = Translator()

# Inicializar el pipeline fill-mask para español (para obtener sinónimos)
fill_mask = pipeline("fill-mask", model="dccuchile/bert-base-spanish-wwm-cased")

# --- Funciones de traducción usando googletrans ---
def traducir_es_a_en(texto):
    return translator.translate(texto, src='es', dest='en').text

def traducir_en_a_es(texto):
    return translator.translate(texto, src='en', dest='es').text

# --- Funciones para obtener variaciones automáticas ---
def obtener_variaciones_automaticas(palabra, top_k=3):
    """
    Usa el pipeline fill-mask para obtener sinónimos de una palabra.
    """
    template = f"El sinónimo de {palabra} es [MASK]."
    predictions = fill_mask(template, top_k=top_k)
    variaciones = []
    for pred in predictions:
        token = pred['token_str'].strip()
        if token.lower() != palabra.lower() and token.isalpha():
            variaciones.append(token)
    return variaciones

def generar_variaciones_automaticas(frase):
    """
    Para cada token relevante (VERB, NOUN, ADP) en la frase,
    se obtienen sinónimos y se generan nuevas versiones de la frase.
    """
    doc = nlp(frase)
    variaciones = []
    for token in doc:
        if token.pos_ in ["VERB", "NOUN", "ADP"]:
            sinonimos = obtener_variaciones_automaticas(token.text, top_k=3)
            for sinonimo in sinonimos:
                nueva_frase = frase.replace(token.text, sinonimo, 1)
                variaciones.append(nueva_frase)
    return variaciones

# --- Función para generar paráfrasis usando Parrot ---
def generar_parafraseos_con_parrot(frase_es):
    """
    Traduce la frase de español a inglés, genera paráfrasis con Parrot,
    y luego traduce las paráfrasis resultantes de vuelta a español.
    """
    # Traducir la frase al inglés
    frase_en = traducir_es_a_en(frase_es)
    # Generar paráfrasis con Parrot
    parafraseos = parrot.augment(
        input_phrase=frase_en,
        max_return_phrases=5,
        adequacy_threshold=0.85,
        fluency_threshold=0.85
    )
    resultados = []
    for item in parafraseos:
        if isinstance(item, tuple) and len(item) >= 1:
            texto_en = item[0]
        elif isinstance(item, str):
            texto_en = item
        else:
            continue
        # Traducir la paráfrasis de vuelta a español
        texto_es = traducir_en_a_es(texto_en)
        resultados.append(texto_es)
    return resultados


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Device set to use cpu


In [5]:

# --- Bloque principal ---
# Frase de entrada en español
frase_es = "Quiero reservar un vuelo a Nueva York el próximo sábado."

# Generar paráfrasis usando Parrot (a través del traductor de Google)
parafraseos_parrot = generar_parafraseos_con_parrot(frase_es)

# Generar variaciones automáticas a partir de la frase original
variaciones_automaticas = generar_variaciones_automaticas(frase_es)

# Combinar ambos resultados y eliminar duplicados
todos_resultados = list(set(parafraseos_parrot + variaciones_automaticas))
todos_resultados = todos_resultados[:10]  # Limitar a 10 resultados

# Mostrar resultados finales
print("\nParáfrasis y variaciones en español:")
for i, r in enumerate(todos_resultados, 1):
    print(f"{i}. {r}")



Paráfrasis y variaciones en español:
1. Quiero reservar un vuelo a Nueva York el próximo viernes.
2. Quiero reservar un vuelo a Nueva York el próximo domingo.
3. Quiero reservar un vuelo a Nueva York el próximo sábado
4. Quiero reservbr un vuelo a Nueva York el próximo sábado.
5. Quiero reservar un mariposa a Nueva York el próximo sábado.
